In [1]:
# Install TensorFlow Hub
!pip install tensorflow_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 KB 17.3 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.



## Scientific Explanation of Image Processing

This notebook leverages TensorFlow and TensorFlow Hub to perform image processing tasks, particularly focusing on enhancing image resolution. The process involves:

1. **Preprocessing**: Adjusting image size and format to meet model requirements.
2. **Model Application**: Utilizing pre-trained models from TensorFlow Hub to enhance or modify images.
3. **Post-processing**: Saving the enhanced images and performing any final adjustments.

These steps are crucial for tasks such as super-resolution, where the goal is to increase the resolution of input images using deep learning techniques.


In [2]:
# Import necessary libraries
import os
import time
from PIL import *
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

2022-11-21 21:25:32.104755: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 21:25:32.313812: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-21 21:25:32.313846: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-21 21:25:32.345215: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-21 21:25:34.235361: W tensorflow/stream_executor/platform/de

In [3]:
# Define a function to save images
# Define a function to preprocess images to make them model-ready
def preprocess_image(image_path):
  """ Loads image from path and preprocesses to make it model ready
      Args:
        image_path: Path to the image file
  """
  hr_image = tf.image.decode_image(tf.io.read_file(image_path))
  # If PNG, remove the alpha channel. The model only supports
  # images with 3 color channels.
  if hr_image.shape[-1] == 4:
    hr_image = hr_image[...,:-1]
  hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
  hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
  hr_image = tf.cast(hr_image, tf.float32)
  return tf.expand_dims(hr_image, 0)

def save_image(image, filename):
  """
    Saves unscaled Tensor Images.
    Args:
      image: 3D image tensor. [height, width, channels]
      filename: Name of the file to save.
  """
  if not isinstance(image, Image.Image):
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  image.save("%s.jpg" % filename)
  print("Saved as %s.jpg" % filename)

In [4]:
# Declaring Constants
IMAGE_PATH = "/work/temp.jpg"
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
model = hub.load(SAVED_MODEL_PATH)

Downloaded https://tfhub.dev/captain-pool/esrgan-tf2/1, Total size: 20.60MB

2022-11-21 21:25:38.449263: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-21 21:25:38.449323: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-21 21:25:38.449350: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-0449f7a0-0404-4608-a8fb-4d9687970365): /proc/driver/nvidia/version does not exist
2022-11-21 21:25:38.449679: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45215 sha256=dd0cd695077228089958db52ae54084cb85c8fb7f1b9e702d10b6f69535b6224
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [6]:
# Import necessary libraries
import anvil.server
import io

anvil.server.connect("67FM7PLGW7MWMUJRPBHGN4XM-CZ27UJXWRRE4WPXP")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [7]:
# Import necessary libraries
import anvil.server
import anvil.media
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import save_img
from tensorflow.keras.preprocessing.image import img_to_array

anvil.server.connect("67FM7PLGW7MWMUJRPBHGN4XM-CZ27UJXWRRE4WPXP")

def write_a_file(file):
  with open('temp.jpg','wb') as f:
    f.write(file)

@anvil.server.callable
def super(file):
  print("Call")
  with anvil.media.TempFile(file) as f:
    img = load_img(f)
    MAX_SIZE = (96, 96)
    img.thumbnail(MAX_SIZE)
    img_array = img_to_array(img)
    save_img('temp.jpg', img_array)




    IMAGE_PATH = "/work/temp.jpg"
    hr_image = preprocess_image(IMAGE_PATH)
    fake_image = model(hr_image)
    fake_image = tf.squeeze(fake_image)
    save_image(tf.squeeze(fake_image), filename="Super Resolution")


    super_img = Image.open("/work/Super Resolution.jpg")
    print(type(img), "IMG Loaded")
    
    super_img = Image.open("Super Resolution.jpg")
    name='super_img'
    bs = io.BytesIO()
    super_img.save(bs, format="JPEG")

  return anvil.BlobMedia("image/jpg", bs.getvalue(), name=name)
  


anvil.server.wait_forever()


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0449f7a0-0404-4608-a8fb-4d9687970365' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>